<a href="https://colab.research.google.com/github/NikhitapHegde/final_year_project/blob/main/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import os
import seaborn as sns
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, Reshape, Input, Conv2DTranspose
from keras.layers import Activation, LeakyReLU, BatchNormalization, Dropout, Resizing
from keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import VGG16

import warnings
warnings.filterwarnings('ignore')

try:
    from tensorflow.keras.optimizers import Adam
except:
    from keras.optimizers import Adam

In [2]:
# NOISE_DIM = 100
# BATCH_SIZE = 4
# STEPS_PER_EPOCH = 3750
# EPOCHS = 10
# SEED = 40
# WIDTH, HEIGHT, CHANNELS = 128, 128, 1

# OPTIMIZER = Adam(0.0002, 0.5)

NOISE_DIM = 100
BATCH_SIZE = 6   # Increase slightly from 4 to 8
STEPS_PER_EPOCH = 100  # Adjust based on dataset size
EPOCHS = 20  # Keep the same, increase to 20-30 if needed
SEED = 40  # No change
WIDTH, HEIGHT, CHANNELS = 128, 128, 1  # No change
OPTIMIZER = Adam(0.0002, 0.5)  # No change

In [3]:
from google.colab import files

# Upload the ZIP file
uploaded = files.upload()

Saving archive (1).zip to archive (1).zip


In [7]:
import zipfile
import os

# Define the name of the uploaded ZIP file
zip_file_name = 'archive (1).zip'  # Change this if your file has a different name

# Extract the ZIP file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('extracted_files')

print("Files extracted to: extracted_files/")

Files extracted to: extracted_files/


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import numpy as np
from PIL import Image
from torchvision.utils import save_image
import matplotlib.pyplot as plt

# Directory paths for training data
non_tumor_dir = "extracted_files/brain_tumor_dataset/no"  # Directory for non-tumor images
tumor_dir = "extracted_files/brain_tumor_dataset/yes"  # Directory for tumor images

# Directories for generated images
non_tumor_output_dir = "extracted_files/no"
tumor_output_dir = "extracted_files/yes"

# Ensure output directories exist
os.makedirs(non_tumor_output_dir, exist_ok=True)
os.makedirs(tumor_output_dir, exist_ok=True)

# Image preprocessing (resize and normalize)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Dataset loading for real images (non-tumor and tumor)
non_tumor_dataset = datasets.ImageFolder(root=non_tumor_dir, transform=transform)
tumor_dataset = datasets.ImageFolder(root=tumor_dir, transform=transform)

# Combine the datasets
all_data = non_tumor_dataset + tumor_dataset

# Create DataLoader for loading batches of data
data_loader = torch.utils.data.DataLoader(all_data, batch_size=64, shuffle=True)

# Define the Generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3 * 64 * 64),
            nn.Tanh()
        )

    def forward(self, z):
        z = self.fc(z)
        z = z.view(z.size(0), 3, 64, 64)
        return z

# Define the Discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(3 * 64 * 64, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Initialize the Generator and Discriminator
generator = Generator().cuda()
discriminator = Discriminator().cuda()

# Set up the optimizer and loss function
criterion = nn.BCELoss()
lr = 0.0002
beta1 = 0.5
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

# Labels for the Generator and Discriminator
real_label = 1
fake_label = 0

# Training loop
num_epochs = 50
z_dim = 100  # Latent vector size
fixed_noise = torch.randn(64, z_dim).cuda()  # Generate fixed noise for monitoring progress

for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(data_loader):
        batch_size = real_images.size(0)
        real_images = real_images.cuda()

        # Generate labels for real images
        labels = torch.full((batch_size,), real_label, dtype=torch.float).cuda()

        # Train Discriminator
        optimizer_D.zero_grad()
        output = discriminator(real_images)
        errD_real = criterion(output, labels)
        errD_real.backward()

        noise = torch.randn(batch_size, z_dim).cuda()
        fake_images = generator(noise)
        labels.fill_(fake_label)

        output = discriminator(fake_images.detach())
        errD_fake = criterion(output, labels)
        errD_fake.backward()

        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        labels.fill_(real_label)
        output = discriminator(fake_images)
        errG = criterion(output, labels)
        errG.backward()

        optimizer_G.step()

        # Print training progress
        if i % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(data_loader)}], "
                  f"ErrD: {errD_real.item() + errD_fake.item():.4f}, ErrG: {errG.item():.4f}")

    # Save generated images every epoch
    with torch.no_grad():
        fake_images = generator(fixed_noise).detach().cpu()
        save_image(fake_images, f'generated_images_epoch_{epoch}.jpg', nrow=8, normalize=True)

    # Save the generated images in the respective directories
    if epoch % 5 == 0:
        # Generate images for non-tumor and tumor categories
        non_tumor_images = generator(torch.randn(1500, z_dim).cuda()).detach().cpu()
        tumor_images = generator(torch.randn(1500, z_dim).cuda()).detach().cpu()

        # Save non-tumor images
        for idx, img in enumerate(non_tumor_images):
            img_path = os.path.join(non_tumor_output_dir, f"non_tumor_{epoch}_{idx}.jpg")
            save_image(img, img_path, normalize=True)

        # Save tumor images
        for idx, img in enumerate(tumor_images):
            img_path = os.path.join(tumor_output_dir, f"tumor_{epoch}_{idx}.jpg")
            save_image(img, img_path, normalize=True)

    # Optionally visualize some of the generated images
    if epoch % 10 == 0:
        plt.figure(figsize=(8, 8))
        plt.imshow(np.transpose(fake_images[0], (1, 2, 0)))
        plt.show()


FileNotFoundError: Couldn't find any class folder in extracted_files/brain_tumor_dataset/no.